In [3]:
import time
import os
import sys
import csv 

import numpy as np
import pandas as pd
from PIL import Image
import cv2
import numpy as np

In [4]:
videos_dir = '../datasets/new_segmented'
open_face_dir = "D:/Software/reluu_extra_space/CMU_MOSEI/Raw/Videos/Full/OpenFace2.0"
mosei_labels = "mosei_labels.csv"
mosei_dataset = "mosei_dataset.csv"
mosei_dataset_int = "mosei_dataset_int.csv"
save_frames_dir = "../labeled_data/emotions_v2"

x_samples = [' x_0', ' x_1', ' x_2', ' x_3', ' x_4', ' x_5', ' x_6', ' x_7', ' x_8', ' x_9', ' x_10', ' x_11', ' x_12', ' x_13', ' x_14', ' x_15', ' x_16', ' x_17', ' x_18', ' x_19', ' x_20', ' x_21', ' x_22', ' x_23', ' x_24', ' x_25', ' x_26', ' x_27', ' x_28', ' x_29', ' x_30', ' x_31', ' x_32', ' x_33', ' x_34', ' x_35', ' x_36', ' x_37', ' x_38', ' x_39', ' x_40', ' x_41', ' x_42', ' x_43', ' x_44', ' x_45', ' x_46', ' x_47', ' x_48', ' x_49', ' x_50', ' x_51', ' x_52', ' x_53', ' x_54', ' x_55', ' x_56', ' x_57', ' x_58', ' x_59', ' x_60', ' x_61', ' x_62', ' x_63', ' x_64', ' x_65', ' x_66', ' x_67']

y_samples = [' y_0', ' y_1', ' y_2', ' y_3', ' y_4', ' y_5', ' y_6', ' y_7', ' y_8', ' y_9', ' y_10', ' y_11', ' y_12', ' y_13', ' y_14', ' y_15', ' y_16', ' y_17', ' y_18', ' y_19', ' y_20', ' y_21', ' y_22', ' y_23', ' y_24', ' y_25', ' y_26', ' y_27', ' y_28', ' y_29', ' y_30', ' y_31', ' y_32', ' y_33', ' y_34', ' y_35', ' y_36', ' y_37', ' y_38', ' y_39', ' y_40', ' y_41', ' y_42', ' y_43', ' y_44', ' y_45', ' y_46', ' y_47', ' y_48', ' y_49', ' y_50', ' y_51', ' y_52', ' y_53', ' y_54', ' y_55', ' y_56', ' y_57', ' y_58', ' y_59', ' y_60', ' y_61', ' y_62', ' y_63', ' y_64', ' y_65', ' y_66', ' y_67']

In [5]:
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
class ShowData:
    def __init__(self, data_set_path):      
        self.data_frame = pd.DataFrame()
        self.data_frame = pd.read_csv(data_set_path)
    
    def __call__(self):
        return self.data_frame.head()

    def row_value(self, row, value):
        return self.data_frame.loc[(self.data_frame[row]==value)]

    def value_intersection(self, row_identifier, row_value, columns):
        return self.data_frame.loc[(self.data_frame[row_identifier]==row_value), columns]

In [6]:
class CreateDataSet:
    '''
    This creates the dataset that can be read via "flow from dataframe" method
    '''
    def __init__(self, videos_dir, open_face_dir, mosei_labels, mosei_dataset, mosei_dataset_int, save_frames_dir, x_samples, y_samples, ShowData:ShowData):
        self.videos_dir = videos_dir
        self.open_face_dir = open_face_dir
        self.mosei_labels = mosei_labels
        self.mosei_dataset = mosei_dataset
        self.mosei_dataset_int = mosei_dataset_int
        self.save_frames_dir = save_frames_dir
        self.x_samples = x_samples
        self.y_samples = y_samples
        self.ShowData = ShowData
        self.headers = ["video_name_segment", "sentiment", "happy", "sad", "anger", "surprise", "disgust", "fear"]
        with open(self.mosei_dataset, "w", newline="") as csvfile:
            csvfile = csv.writer(csvfile)
            csvfile.writerow(self.headers)
        with open(self.mosei_dataset_int, "w", newline="") as csvfile_int:
            csvfile_int = csv.writer(csvfile_int)
            csvfile_int.writerow(self.headers)


    def crop_image(self, image, frame, open_face_df, debug1, debug2):
        '''
        This opens the open_face2.0 csv file that contains the face coordinates, crops them, returns the cropped image
        ''' 
        try:
            x_values = open_face_df.value_intersection("frame", frame, self.x_samples)
            y_values = open_face_df.value_intersection("frame", frame, self.y_samples)
            x_max, x_min = int(max(x_values.values[0])), int(min(x_values.values[0]))
            y_max, y_min = int(max(y_values.values[0])), int(min(y_values.values[0]))
            # li = [x_max!=0, x_min!=0, y_max!=0, y_min!=0]
            # print(any(li))
            if all([x_max!=0, x_min!=0, y_max!=0, y_min!=0]):
                image = image[y_min:y_max, x_min:x_max]
                return image
            else:
                return None
        except:
            print(frame)
            print(f"raw_name: {debug1}, segmented video {debug2}")


    def gogo(self):
        '''
        This crops the videos and creates the csv dataset. if the class flag for pares_videos=False then it will only make a new dataset

        mosei_csv = [0="raw_name", 1="parent_video", 2="segmented_video", 3="segment_number", 4="start", 5="end", 6="start_frame", 7="end_frame",                        8="sentiment",9="happy", 10="sad", 11="anger", 12="surprise", 13="disgust", 14="fear"]
        '''
        with open(mosei_labels, "r") as csvfile:
            csvfile = csv.reader(csvfile, delimiter=",")
            next(csvfile)
            with open(self.mosei_dataset, "a", newline="") as csvappend:
                csvappend = csv.writer(csvappend)
                with open(self.mosei_dataset_int, "a", newline="") as csvappend_int:
                    csvappend_int = csv.writer(csvappend_int)
                    for row in csvfile:
                        if "ChhZna-aBK4" == row[1]:
                            break
                        else:
                            current_frame = int(row[6])+1
                            end_frame = int(row[7])
                            mp4 = f"{row[2]}.mp4"
                            video_file = os.path.join(videos_dir, mp4)
                            video_cap = cv2.VideoCapture(video_file)
                            success,image = video_cap.read()
                            parent_video_csv = f"{row[1]}.csv"
                            open_face_csv = os.path.join(self.open_face_dir, parent_video_csv)
                            open_face_df = self.ShowData(open_face_csv)

                            while current_frame <= end_frame and success:
                                unique_image_name = f"{row[2]}_{current_frame}.jpg"
                                image = np.array(image)
                                image = self.crop_image(image, current_frame, open_face_df, row[0], row[2])
                                if image is not None:
                                    try:
                                        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                                        image = Image.fromarray(image)
                                        image_path = os.path.join(self.save_frames_dir, unique_image_name)
                                        image.save(image_path, "JPEG")

                                        write_row = [unique_image_name, float(row[8]), float(row[9]), float(row[10]), 
                                        float(row[11]), float(row[12]), float(row[13]), float(row[14])]
                                        csvappend.writerow(write_row)

                                        write_row_int = [unique_image_name, int(round(float(row[8]))), int(round(float(row[9]))), 
                                        int(round(float(row[10]))), 
                                        int(round(float(row[11]))), int(round(float(row[12]))), int(round(float(row[13]))), int(round(float(row[14])))]
                                        csvappend_int.writerow(write_row_int)                                      
                                    except:
                                        print(f"Failed to save Image ### name: {unique_image_name}, video_segment: {row[2]}, frame {current_frame}")
                                else:
                                    print(f"{unique_image_name}: has 0 values for the x, y cordinates")
                                current_frame+=1
                                success,image = video_cap.read()

In [8]:
tick = time.clock()
abc = CreateDataSet(videos_dir, open_face_dir, mosei_labels, mosei_dataset, mosei_dataset_int, save_frames_dir, x_samples, y_samples, ShowData)
abc.gogo()
toc = time.clock()
print(f"Runtime: {toc-tick}")

-3nNcZdcdvU_0_1.jpg: has 0 values for the x, y cordinates
-3nNcZdcdvU_0_2.jpg: has 0 values for the x, y cordinates
-3nNcZdcdvU_0_3.jpg: has 0 values for the x, y cordinates
-3nNcZdcdvU_0_4.jpg: has 0 values for the x, y cordinates
-3nNcZdcdvU_0_5.jpg: has 0 values for the x, y cordinates
-3nNcZdcdvU_0_6.jpg: has 0 values for the x, y cordinates
-3nNcZdcdvU_0_7.jpg: has 0 values for the x, y cordinates
-3nNcZdcdvU_0_8.jpg: has 0 values for the x, y cordinates
-3nNcZdcdvU_0_9.jpg: has 0 values for the x, y cordinates
-3nNcZdcdvU_0_10.jpg: has 0 values for the x, y cordinates
-3nNcZdcdvU_0_11.jpg: has 0 values for the x, y cordinates
-3nNcZdcdvU_0_12.jpg: has 0 values for the x, y cordinates
-571d8cVauQ_0_1.jpg: has 0 values for the x, y cordinates
-571d8cVauQ_0_2.jpg: has 0 values for the x, y cordinates
-571d8cVauQ_0_3.jpg: has 0 values for the x, y cordinates
-571d8cVauQ_0_4.jpg: has 0 values for the x, y cordinates
-571d8cVauQ_0_5.jpg: has 0 values for the x, y cordinates
-571d8cVauQ

In [10]:
for root, dirs, files in os.walk("../labeled_data/emotions_v2"):
    with open(mosei_dataset) as f:
        next(f)
        pics = len(files)
        dataset = sum(1 for line in f)
        print(dataset)
        print(pics)
        assert(dataset==pics)
        # 541929 differences

2142198
2142198


In [119]:
def Diff(li1, li2):
    '''
    Returns differences of li1 and li2.
    '''
    return (list(list(set(li1)-set(li2)) + list(set(li2)-set(li1))))

pics = os.listdir("../labeled_data/emotions_v2")
data = pd.read_csv(mosei_dataset)
data.head()
names = data.video_name_segment.tolist()
remove_list = Diff(pics, names)
print(remove_list)

[]


In [118]:
# pic_path = "../labeled_data/emotions_v2"
# for _file in remove_list:
#     pa = os.path.join(pic_path, _file)
#     os.remove(pa)

In [1]:
'''
Delete all files in labeled data folder
'''
# for root, dirs, files in os.walk("../labeled_data/emotions_v2"):
#     for x in files:
#         os.remove(os.path.join(root, x))

In [ ]:
# ['-3g5yACwYnA_3_396.jpg', '-3g5yACwYnA_5_2222.jpg', '--qXJuDtHPw_0_695.jpg', '-Alixo7euuU_10_5732.jpg', '-Alixo7euuU_10_5730.jpg', '-Alixo7euuU_10_5731.jpg', '-Alixo7euuU_10_5729.jpg', '-3g5yACwYnA_0_2482.jpg', '-3g5yACwYnA_1_3597.jpg', '-3g5yACwYnA_2_145.jpg', '-3g5yACwYnA_4_796.jpg', '-Alixo7euuU_10_5728.jpg']

'''
List of names that threw an exception even after we checked to make sure the face boudnaires were correct
name: -Alixo7euuU_10_5733.jpg, video_segment: -Alixo7euuU_10, frame 5733
name: -Alixo7euuU_10_5734.jpg, video_segment: -Alixo7euuU_10, frame 5734
name: -Alixo7euuU_10_5735.jpg, video_segment: -Alixo7euuU_10, frame 5735
name: -Alixo7euuU_10_5736.jpg, video_segment: -Alixo7euuU_10, frame 5736
name: -Alixo7euuU_10_5737.jpg, video_segment: -Alixo7euuU_10, frame 5737
name: -Alixo7euuU_10_5738.jpg, video_segment: -Alixo7euuU_10, frame 5738
name: -Alixo7euuU_10_5739.jpg, video_segment: -Alixo7euuU_10, frame 5739
name: -Alixo7euuU_4_3907.jpg, video_segment: -Alixo7euuU_4, frame 3907
name: -Alixo7euuU_4_3908.jpg, video_segment: -Alixo7euuU_4, frame 3908
name: -Alixo7euuU_4_3909.jpg, video_segment: -Alixo7euuU_4, frame 3909
name: -Alixo7euuU_5_4176.jpg, video_segment: -Alixo7euuU_5, frame 4176
name: -Alixo7euuU_5_4177.jpg, video_segment: -Alixo7euuU_5, frame 4177
name: -Alixo7euuU_5_4178.jpg, video_segment: -Alixo7euuU_5, frame 4178
name: -Alixo7euuU_5_4179.jpg, video_segment: -Alixo7euuU_5, frame 4179
name: -Alixo7euuU_5_4180.jpg, video_segment: -Alixo7euuU_5, frame 4180
name: -Alixo7euuU_5_4181.jpg, video_segment: -Alixo7euuU_5, frame 4181
name: -Alixo7euuU_5_4182.jpg, video_segment: -Alixo7euuU_5, frame 4182
name: -Alixo7euuU_5_4183.jpg, video_segment: -Alixo7euuU_5, frame 4183
name: -Alixo7euuU_5_4184.jpg, video_segment: -Alixo7euuU_5, frame 4184
name: -Alixo7euuU_5_4185.jpg, video_segment: -Alixo7euuU_5, frame 4185
name: -Alixo7euuU_5_4186.jpg, video_segment: -Alixo7euuU_5, frame 4186
name: -Alixo7euuU_5_4187.jpg, video_segment: -Alixo7euuU_5, frame 4187
name: -Alixo7euuU_5_4188.jpg, video_segment: -Alixo7euuU_5, frame 4188
name: -Alixo7euuU_5_4189.jpg, video_segment: -Alixo7euuU_5, frame 4189
name: -Alixo7euuU_5_4190.jpg, video_segment: -Alixo7euuU_5, frame 4190
name: -Alixo7euuU_5_4191.jpg, video_segment: -Alixo7euuU_5, frame 4191
name: -Alixo7euuU_5_4192.jpg, video_segment: -Alixo7euuU_5, frame 4192
name: -Alixo7euuU_5_4193.jpg, video_segment: -Alixo7euuU_5, frame 4193
name: -Alixo7euuU_5_4194.jpg, video_segment: -Alixo7euuU_5, frame 4194
name: -Alixo7euuU_5_4195.jpg, video_segment: -Alixo7euuU_5, frame 4195
name: -Alixo7euuU_5_4196.jpg, video_segment: -Alixo7euuU_5, frame 4196
name: -Alixo7euuU_5_4197.jpg, video_segment: -Alixo7euuU_5, frame 4197
name: -Alixo7euuU_7_4250.jpg, video_segment: -Alixo7euuU_7, frame 4250
name: -Alixo7euuU_7_4251.jpg, video_segment: -Alixo7euuU_7, frame 4251
name: -Alixo7euuU_7_4252.jpg, video_segment: -Alixo7euuU_7, frame 4252
name: -Alixo7euuU_7_4253.jpg, video_segment: -Alixo7euuU_7, frame 4253
name: -Alixo7euuU_7_4254.jpg, video_segment: -Alixo7euuU_7, frame 4254
name: -Alixo7euuU_7_4255.jpg, video_segment: -Alixo7euuU_7, frame 4255
name: -Alixo7euuU_7_4256.jpg, video_segment: -Alixo7euuU_7, frame 4256
name: -Alixo7euuU_7_4257.jpg, video_segment: -Alixo7euuU_7, frame 4257
name: -Alixo7euuU_7_4258.jpg, video_segment: -Alixo7euuU_7, frame 4258
name: -Alixo7euuU_7_4259.jpg, video_segment: -Alixo7euuU_7, frame 4259
name: -Alixo7euuU_7_4260.jpg, video_segment: -Alixo7euuU_7, frame 4260
name: -Alixo7euuU_7_4261.jpg, video_segment: -Alixo7euuU_7, frame 4261
name: -Alixo7euuU_7_4262.jpg, video_segment: -Alixo7euuU_7, frame 4262
name: -Alixo7euuU_7_4263.jpg, video_segment: -Alixo7euuU_7, frame 4263
name: -Alixo7euuU_7_4264.jpg, video_segment: -Alixo7euuU_7, frame 4264
name: -Alixo7euuU_7_4265.jpg, video_segment: -Alixo7euuU_7, frame 4265
name: -Alixo7euuU_7_4266.jpg, video_segment: -Alixo7euuU_7, frame 4266
name: -Alixo7euuU_7_4267.jpg, video_segment: -Alixo7euuU_7, frame 4267
name: -Alixo7euuU_7_4268.jpg, video_segment: -Alixo7euuU_7, frame 4268
name: -Alixo7euuU_7_4269.jpg, video_segment: -Alixo7euuU_7, frame 4269
name: -Alixo7euuU_7_4270.jpg, video_segment: -Alixo7euuU_7, frame 4270
name: -Alixo7euuU_7_4271.jpg, video_segment: -Alixo7euuU_7, frame 4271
name: -Alixo7euuU_7_4436.jpg, video_segment: -Alixo7euuU_7, frame 4436
name: -Alixo7euuU_7_4437.jpg, video_segment: -Alixo7euuU_7, frame 4437
name: -Alixo7euuU_7_4438.jpg, video_segment: -Alixo7euuU_7, frame 4438
name: -Alixo7euuU_7_4439.jpg, video_segment: -Alixo7euuU_7, frame 4439
name: -Alixo7euuU_7_4440.jpg, video_segment: -Alixo7euuU_7, frame 4440
name: -Alixo7euuU_7_4441.jpg, video_segment: -Alixo7euuU_7, frame 4441
name: -Alixo7euuU_7_4442.jpg, video_segment: -Alixo7euuU_7, frame 4442
name: -Alixo7euuU_7_4443.jpg, video_segment: -Alixo7euuU_7, frame 4443
name: -Alixo7euuU_7_4444.jpg, video_segment: -Alixo7euuU_7, frame 4444
name: -Alixo7euuU_7_4445.jpg, video_segment: -Alixo7euuU_7, frame 4445
name: -Alixo7euuU_7_4446.jpg, video_segment: -Alixo7euuU_7, frame 4446
name: -Alixo7euuU_7_4447.jpg, video_segment: -Alixo7euuU_7, frame 4447
name: -Alixo7euuU_7_4448.jpg, video_segment: -Alixo7euuU_7, frame 4448
name: -Alixo7euuU_7_4449.jpg, video_segment: -Alixo7euuU_7, frame 4449
name: -Alixo7euuU_7_4450.jpg, video_segment: -Alixo7euuU_7, frame 4450
name: -Alixo7euuU_7_4451.jpg, video_segment: -Alixo7euuU_7, frame 4451
name: -Alixo7euuU_7_4452.jpg, video_segment: -Alixo7euuU_7, frame 4452
name: -Alixo7euuU_7_4453.jpg, video_segment: -Alixo7euuU_7, frame 4453
name: -Alixo7euuU_7_4454.jpg, video_segment: -Alixo7euuU_7, frame 4454
name: -Alixo7euuU_7_4455.jpg, video_segment: -Alixo7euuU_7, frame 4455
name: -Alixo7euuU_7_4456.jpg, video_segment: -Alixo7euuU_7, frame 4456
name: -Alixo7euuU_7_4457.jpg, video_segment: -Alixo7euuU_7, frame 4457
name: -Alixo7euuU_7_4458.jpg, video_segment: -Alixo7euuU_7, frame 4458
name: -Alixo7euuU_7_4459.jpg, video_segment: -Alixo7euuU_7, frame 4459
name: -Alixo7euuU_7_4460.jpg, video_segment: -Alixo7euuU_7, frame 4460
name: -Alixo7euuU_7_4461.jpg, video_segment: -Alixo7euuU_7, frame 4461
name: -Alixo7euuU_7_4462.jpg, video_segment: -Alixo7euuU_7, frame 4462
name: -Alixo7euuU_7_4463.jpg, video_segment: -Alixo7euuU_7, frame 4463
name: -Alixo7euuU_7_4464.jpg, video_segment: -Alixo7euuU_7, frame 4464
name: -Alixo7euuU_7_4465.jpg, video_segment: -Alixo7euuU_7, frame 4465
name: -Alixo7euuU_7_4466.jpg, video_segment: -Alixo7euuU_7, frame 4466
name: -Alixo7euuU_7_4467.jpg, video_segment: -Alixo7euuU_7, frame 4467
name: -Alixo7euuU_7_4468.jpg, video_segment: -Alixo7euuU_7, frame 4468
name: -Alixo7euuU_7_4469.jpg, video_segment: -Alixo7euuU_7, frame 4469
name: -Alixo7euuU_7_4470.jpg, video_segment: -Alixo7euuU_7, frame 4470
name: -Alixo7euuU_7_4471.jpg, video_segment: -Alixo7euuU_7, frame 4471
name: -Alixo7euuU_7_4472.jpg, video_segment: -Alixo7euuU_7, frame 4472
name: -Alixo7euuU_7_4473.jpg, video_segment: -Alixo7euuU_7, frame 4473
name: -Alixo7euuU_7_4474.jpg, video_segment: -Alixo7euuU_7, frame 4474
name: -Alixo7euuU_7_4475.jpg, video_segment: -Alixo7euuU_7, frame 4475
name: -Alixo7euuU_7_4476.jpg, video_segment: -Alixo7euuU_7, frame 4476
name: -Alixo7euuU_7_4477.jpg, video_segment: -Alixo7euuU_7, frame 4477
name: -Alixo7euuU_7_4478.jpg, video_segment: -Alixo7euuU_7, frame 4478
name: -Alixo7euuU_7_4479.jpg, video_segment: -Alixo7euuU_7, frame 4479
name: -Alixo7euuU_9_5605.jpg, video_segment: -Alixo7euuU_9, frame 5605
name: -Alixo7euuU_9_5606.jpg, video_segment: -Alixo7euuU_9, frame 5606
name: -Alixo7euuU_9_5607.jpg, video_segment: -Alixo7euuU_9, frame 5607
name: -HwX2H8Z4hY_0_6.jpg, video_segment: -HwX2H8Z4hY_0, frame 6
name: -HwX2H8Z4hY_0_8.jpg, video_segment: -HwX2H8Z4hY_0, frame 8
name: -HwX2H8Z4hY_1_82.jpg, video_segment: -HwX2H8Z4hY_1, frame 82
'''